## Let's try RNN

In [136]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os


In [145]:
DATA_DIR = "./generated_data"
CSV_NAME = "generated_data_2.csv"

In [146]:
df = pd.read_csv(os.path.join(DATA_DIR, CSV_NAME))
df = df.interpolate() # to prevent nan in model output
df.head()


,noisy motion,kalman prediction,ground truth,motion noise stdev,laser noise stdev,laser range 1,laser range 2,laser range 3,laser range 4,laser range 5,laser range 6
0,32.063360,5.014954,4.527978,2,0.1,4.998537,4.927686,4.919777,5.089352,4.958714,4.836348
1,32.108053,5.158915,4.574486,2,0.1,5.013025,4.986345,4.975599,4.671942,4.971018,4.949959
2,32.097932,5.319893,4.564363,2,0.1,5.034334,4.927298,5.154497,5.088942,4.825726,4.899361
3,32.377241,5.193629,4.765297,2,0.1,4.656789,4.656965,4.581776,4.857603,4.824811,4.783723
4,31.402264,5.363585,5.098723,2,0.1,4.565324,4.552719,4.370080,4.386198,4.455054,4.489313


In [138]:
# Define the RNN model
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        if out.isnan().any():
            print("NAN:", out)
        # print(out.shape)
        out = self.fc(out[:, -1, :]) # taking only the last hidden layer output
        if out.isnan().any():
            print("NAN: Second: ", out)
        # print(out.shape)
        return out

In [139]:
# Create some dummy data
seq_len = 1
batch_size = 5
# num_batches = 69
num_batches = len(df) // (seq_len)
input_size = 9
output_size = 1

In [140]:
labels = np.array(df["ground truth"].to_list())
input_data_df = df[['noisy motion', 'motion noise stdev', 'laser noise stdev', 'laser range 1',
       'laser range 2', 'laser range 3', 'laser range 4', 'laser range 5',
       'laser range 6']].to_numpy()

input_data = input_data_df[:num_batches * seq_len]
input_data = torch.from_numpy(np.reshape(input_data, (num_batches, seq_len, input_data.shape[-1]))).type(torch.float32)

output_data = labels[:num_batches * seq_len]
output_data = torch.from_numpy(np.reshape(output_data, (num_batches, -1))).type(torch.float32)


In [141]:
dataset = torch.utils.data.TensorDataset(input_data, output_data)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [142]:
embedding_size = 9
hidden_size = 64
output_size = 1
model = SimpleRNN(embedding_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [144]:
# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    for batch_inputs, batch_outputs in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_outputs)
        loss.backward()
        optimizer.step()

    # Print the loss after every epoch
    if epoch %10 ==0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))



Epoch [1/100], Loss: 0.0684
Epoch [11/100], Loss: 0.0384
Epoch [21/100], Loss: 0.0176
Epoch [31/100], Loss: 0.0020
Epoch [41/100], Loss: 0.0013
Epoch [51/100], Loss: 0.0049
Epoch [61/100], Loss: 0.5213
Epoch [71/100], Loss: 0.0097
Epoch [81/100], Loss: 0.0054
Epoch [91/100], Loss: 0.0224
